In [15]:
import pandas as pd
dataset = pd.read_csv("../dataset/student-mat.csv", delimiter=";")

In [51]:
# list(dataset)
# print(dataset.loc[:,["goout", "studytime", "G3"]])
print(dataset["studytime"])

0      2
1      2
2      2
3      3
4      2
5      2
6      2
7      2
8      2
9      2
10     2
11     3
12     1
13     2
14     3
15     1
16     3
17     2
18     1
19     1
20     2
21     1
22     2
23     2
24     3
25     1
26     1
27     1
28     2
29     2
      ..
365    2
366    3
367    1
368    1
369    2
370    2
371    1
372    3
373    1
374    3
375    3
376    3
377    2
378    2
379    2
380    2
381    1
382    2
383    1
384    1
385    3
386    1
387    3
388    2
389    2
390    2
391    1
392    1
393    1
394    1
Name: studytime, Length: 395, dtype: int64


In [64]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import SingleIntervalTicker
p = figure(title="Studies",
          plot_width=400, plot_height=400)
p.circle(dataset["studytime"], dataset["G3"],
         fill_alpha=0.1, size=15
        )
p.xaxis.ticker = SingleIntervalTicker(interval=1)
output_notebook()
show(p)

Loading BokehJS ...